In [13]:
from dotenv import load_dotenv
import os
import langchain
from langchain_google_genai import ChatGoogleGenerativeAI


## API Key Setup and Verification

In [8]:
load_dotenv()

True

In [14]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llm_response = llm.invoke("Tell me a joke")
print(llm_response)


content="Why don't scientists trust atoms? \n\nBecause they make up everything!" additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--e94e37cf-7ae4-4d5c-b58a-e896631e76da-0' usage_metadata={'input_tokens': 4, 'output_tokens': 17, 'total_tokens': 21, 'input_token_details': {'cache_read': 0}}


## Output Parser

In [15]:
from langchain_core.output_parsers import StrOutputParser

In [16]:
output_parser = StrOutputParser()
output_parser.invoke(llm_response)

"Why don't scientists trust atoms? \n\nBecause they make up everything!"

## Simple Chain


In [17]:
chain = llm | output_parser

In [18]:
chain.invoke("Tell me something about sun")

"The Sun is a G-type main-sequence star,  meaning it generates energy through nuclear fusion of hydrogen into helium in its core.  This process releases enormous amounts of energy in the form of light and heat, which is vital for life on Earth.  It's about 93 million miles (150 million kilometers) away from Earth and makes up about 99.86% of the total mass of the Solar System."

## Structured Output


In [19]:
from typing import List
from pydantic import BaseModel, Field

class MobileReview(BaseModel):
    phone_model: str = Field(description="Name and model of the phone  '\n'")
    rating: float = Field(description="Overall rating out of 5")
    pros: List[str] = Field(description="List of positive aspects")
    cons: List[str] = Field(description="List of negative aspects")
    summary: str = Field(description="Brief summary of the review")

review_text = """
Just got my hands on the new Galaxy S21 and wow, this thing is slick! The screen is gorgeous,
colors pop like crazy. Camera's insane too, especially at night - my Insta game's never been
stronger. Battery life's solid, lasts me all day no problem.
Not gonna lie though, it's pretty pricey. And what's with ditching the charger? C'mon Samsung.
Also, still getting used to the new button layout, keep hitting Bixby by mistake.
Overall, I'd say it's a solid 4 out of 5. Great phone, but a few annoying quirks keep it from
being perfect. If you're due for an upgrade, definitely worth checking out!
"""

structured_llm = llm.with_structured_output(MobileReview)
output = structured_llm.invoke(review_text)
output


MobileReview(phone_model='Galaxy S21', rating=4.0, pros=['Gorgeous screen', 'Insane camera, especially at night', 'Solid battery life'], cons=['Pricey', 'No charger included', 'New button layout'], summary='Great phone, but a few annoying quirks keep it from being perfect.')

In [20]:
output.pros

['Gorgeous screen', 'Insane camera, especially at night', 'Solid battery life']

## Prompt Template

In [21]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
chain = prompt | llm | output_parser
result = chain.invoke({"topic": "programming"})
print(result)


Why do programmers prefer dark mode?  Because light attracts bugs!


## LLM Messages

In [22]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a helpful assistant that tells jokes."),
    HumanMessage(content="Tell me about programming")
]
response = llm.invoke(messages)
print(response.content)


Why do programmers prefer dark mode?

Because light attracts bugs!


In [23]:

template = ChatPromptTemplate([
    ("system", "You are a helpful assistant that tells jokes."),
    ("human", "Tell me about {topic}")
])
chain = template | llm
response = chain.invoke({"topic": "cars"})
print(response.content)


Why did the car get a flat tire?  Because it ran over a bunch of nails... and they were totally hammered!


## Document Processing for RAG Systems

### 1. Loading Documents

In [24]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document
import os

def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

folder_path = "contents/docs/"
documents = load_documents(folder_path)
print(f"Loaded {len(documents)} documents from the folder.")


Unsupported file type: .DS_Store
Loaded 5 documents from the folder.


### 2. Splitting Documents

In [25]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")


Split the documents into 49 chunks.


### 3. Creating google genAi Embeddings for RAG Systems

In [26]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_function = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
document_embeddings = embedding_function.embed_documents([split.page_content for split in splits])
print(f"Created embeddings for {len(document_embeddings)} document chunks.")

Created embeddings for 49 document chunks.


## Vector Store for RAG system

In [27]:
from langchain_chroma import Chroma

collection_name = "my_collection"
vectorstore = Chroma.from_documents(
    collection_name=collection_name,
    documents=splits,
    embedding=embedding_function,
    persist_directory="./chroma_db"
)
print("Vector store created and persisted to './chroma_db'")


Vector store created and persisted to './chroma_db'


## Performing similarity Search

In [28]:
query = "What are the types of cars"
search_results = vectorstore.similarity_search(query, k=2)
print(f"\nTop 2 most relevant chunks for the query: '{query}'\n")
for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")
    print()



Top 2 most relevant chunks for the query: 'What are the types of cars'

Result 1:
Source: contents/docs/Types_of_Cars.docx
Content: Types of Cars

Cars can be categorized by body style. Sedans are four-door vehicles offering comfort and practicality. Coupes are sportier, while hatchbacks provide flexible cargo space. SUVs dominate global markets for their size and versatility. Convertibles emphasize style and leisure, and trucks are valued for utility and towing capabilities.

Fuel type also defines vehicles. Gasoline cars remain common due to infrastructure. Diesel engines provide torque and efficiency, especially for trucks. Hybrids combine combustion engines with electric motors to balance efficiency. Electric vehicles are gaining adoption, offering zero tailpipe emissions. Hydrogen fuel cell cars, though rare, represent another pathway.

Result 2:
Source: contents/docs/Types_of_Cars.docx
Content: Cars can be categorized by body style. Sedans are four-door vehicles offering comfort

### Creating a Retriever

In [29]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
retriever_results = retriever.invoke("What are the types of cars")
print(retriever_results)


[Document(id='75593228-0a8e-4284-bf5c-dd6334807874', metadata={'source': 'contents/docs/Types_of_Cars.docx'}, page_content='Types of Cars\n\nCars can be categorized by body style. Sedans are four-door vehicles offering comfort and practicality. Coupes are sportier, while hatchbacks provide flexible cargo space. SUVs dominate global markets for their size and versatility. Convertibles emphasize style and leisure, and trucks are valued for utility and towing capabilities.\n\nFuel type also defines vehicles. Gasoline cars remain common due to infrastructure. Diesel engines provide torque and efficiency, especially for trucks. Hybrids combine combustion engines with electric motors to balance efficiency. Electric vehicles are gaining adoption, offering zero tailpipe emissions. Hydrogen fuel cell cars, though rare, represent another pathway.'), Document(id='13aa8c9a-7fb9-4202-a0aa-e25091349bd5', metadata={'source': 'contents/docs/Types_of_Cars.docx'}, page_content='Cars can be categorized b

## Building a RAG Chain

In [30]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = """Answer the question based only on the following context:
{context}
Question: {question}
Answer: """

prompt = ChatPromptTemplate.from_template(template)

def docs2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


## Using RAG Chain

In [31]:
question = "What are the types of cars?"
response = rag_chain.invoke(question)
print(f"Question: {question}")
print(f"Answer: {response}")


Question: What are the types of cars?
Answer: Based on the provided text, cars can be categorized by body style (sedans, coupes, hatchbacks, SUVs, convertibles, trucks) and fuel type (gasoline, diesel, hybrid, electric, hydrogen fuel cell).


# Conversational RAG 

## Handling Follow-Up Questions

In [37]:
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


contextualize_q_system_prompt = """
Given a chat history and the latest user question
which might reference context in the chat history,
formulate a standalone question which can be understood
without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is.
"""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

contextualize_chain = contextualize_q_prompt | llm | StrOutputParser()
print(contextualize_chain.invoke({"input": "Tell its strengths and weaknesses", "chat_history": []}))


What are its strengths and weaknesses?


In [38]:
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the following context to answer the user's question."),
    ("system", "Context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [40]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []
question1 = "What are the types of cars?"
answer1 = rag_chain.invoke({"input": question1, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question1),
    AIMessage(content=answer1)
])

print(f"Human: {question1}")
print(f"AI: {answer1}\n")

question2 = "Tell its strengths and weaknesses"
answer2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question2),
    AIMessage(content=answer2)
])

print(f"Human: {question2}")
print(f"AI: {answer2}")


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 53
}
].


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 51
}
]